In [1]:
from typing import Any, Tuple

import math

import salabim as sim

env = sim.Environment(time_unit='seconds')
env.animation_parameters(True, speed=60)
customer_to_log = 500
customer_speed = 30 #units/second, 1 unit is 5 cm. 20 = 1 m/s = 3.6 km/h
# Creating "supermarket" queue to monitor customer population and stay
supermarket_queue = sim.Queue('Supermarket')

#Shopping baskets and carts:
carts = env.Resource('carts', capacity=45) #45
baskets = env.Resource('baskets', capacity=500)#several hundred
cart_basket_distribution = sim.Pdf((carts, 0.8,baskets, 0.2))
customer_basketcart_distribution_monitor = sim.Monitor("Customer basket and cart monitor")

#Clerks
#Bread and cheese use resources as they are working via counters. Checkout likely has to use queue as the customers need to choose the smallest one
bread_clerks = env.Resource('bread_clerks', capacity=4) #4 employees, 1-6 items takes 2 min
bread_time_distribution = sim.Exponential(2*60)
cheese_and_dairy_clerks = env.Resource('cheese_and_dairy_clerks', capacity=3) #3 employees, 1 min avg.
cheese_and_dairy_time_distribution = sim.Exponential(1*60)

# Department queues for monitors/animation
departments = {
    "fruit_and_vegetables": sim.Queue("Fruit and Vegetables"),
    "meat_and_fish": sim.Queue("Meat and Fish"),
    "bread": sim.Queue("Bread"),
    "cheese_and_dairy": sim.Queue("Cheese and Dairy"),
    "canned_and_packed_food": sim.Queue("Canned and Packed Food"),
    "frozen_foods": sim.Queue("Frozen Foods"),
    "drinks": sim.Queue("Drinks"),
}

#Checkouts
number_of_checkouts = 3
time_per_item_distribution = sim.Exponential(1.1)
payment_time_distribution = sim.Uniform(40, 60)

#Distributions of items per customer
fruit_and_vegetables_distribution = sim.Triangular(4, 22,10)  # min=4, mode=10, max=22
meat_and_fish_distribution = sim.Triangular(0, 9, 4)           # min=0, mode=4, max=9
bread_distribution = sim.Triangular(1, 10, 4)                  # min=1, mode=4, max=10
cheese_and_dairy_distribution = sim.Triangular(1, 11, 3)       # min=1, mode=3, max=11
canned_and_packed_food_distribution = sim.Triangular(6, 35, 17)# min=6, mode=17, max=35
frozen_foods_distribution = sim.Triangular(2, 19, 8)           # min=2, mode=8, max=19
drinks_distribution = sim.Triangular(1, 20, 9)                 # min=1, mode=9, max=20

item_taking_distribution = sim.Uniform(20,30) #Time to take item out of shelf

#Route choice distribution
#Routes
route1 = [
"fruit_and_vegetables",
"meat_and_fish",
"bread",
"cheese_and_dairy",
"canned_and_packed_food",
"frozen_foods",
"drinks",
] #ABCDEF, 80% 
route2 = [
"meat_and_fish",
"bread",
"cheese_and_dairy",
"fruit_and_vegetables",
"canned_and_packed_food",
"frozen_foods",
"drinks",
] #BCDAEFG, 20%
route_distribution = sim.Pdf((route1, 0.8, route2, 0.2))

class Customer(sim.Component):
    """
    Customer class for the supermarket. 
    Traverses the store to via its route to fulfill its shopping_list, while carrying either a shopping basket or cart.
    """
    def setup(self):
        self.route = route_distribution.sample()
        self.shopping_list = {
            "fruit_and_vegetables": int(round(fruit_and_vegetables_distribution.sample())),
            "meat_and_fish": int(round(meat_and_fish_distribution.sample())),
            "bread": int(round(bread_distribution.sample())),
            "cheese_and_dairy": int(round(cheese_and_dairy_distribution.sample())),
            "canned_and_packed_food": int(round(canned_and_packed_food_distribution.sample())),
            "frozen_foods": int(round(frozen_foods_distribution.sample())),
            "drinks": int(round(drinks_distribution.sample())),
        }
        self.carrying = None
        self.actions_log = []
        self.traj = sim.TrajectoryPolygon(polygon=(routing_options['entry']), vmax=customer_speed)
        self.animation_representation = sim.AnimateCircle(10, 5, 0, 360, False, #Ellips 
        x=lambda t: self.traj.x(t),
        y=lambda t: self.traj.y(t),
        angle=lambda t: self.traj.angle(t)) 
        #location changes based on trajectory
        

    def log_action(self, action):
        """Helper function to log an action with the current time."""
        self.actions_log.append((env.now(), action))
        
    def animation_objects(self, id=None):
        return 10, 15, self.animation_representation #some spacing on all sides
        
        
    def move_to(self, location):
        """
        Helper function to create trajectories to a given location for animation purposes.
        
        If location is iterable, it will be used to as a series of x,y coordinate pairs to traverse
        If location is a string, it represents a department and routing will be looked up in routing_options
        
        """
        self.animation_representation.visible = True
        self.animation_representation.show()
        
        current_x = self.traj.x(env.now())
        current_y = self.traj.y(env.now())

        if location == "bread":
            queue_tip = 100+8*40 - bread_clerks.requesters().length()*10
            self.traj = sim.TrajectoryPolygon(polygon = (current_x, current_y, 100+2.5*40, queue_tip), vmax=customer_speed)
        elif location == "cheese_and_dairy":
            queue_tip = 100+8*40 - cheese_and_dairy_clerks.requesters().length()*10
            self.traj = sim.TrajectoryPolygon(polygon = (current_x, current_y, 100+2.5*40, queue_tip), vmax=customer_speed)
        elif type(location) is list or type(location) is tuple:
            self.traj = sim.TrajectoryPolygon(polygon=(current_x, current_y, *location), vmax=customer_speed)
        else:
            self.traj = sim.TrajectoryPolygon(polygon=(current_x, current_y, *routing_options[location]), vmax=customer_speed) 
        
    def process(self):
        """"
        Process determines what the customer will do. At the start they will take a cart or basket. Afterwards they will traverse their route and take the items they need according to their shopping list. If they have finished their route (when progress is equal to the length of the shopping list), they will go to the checkout.
        """
        self.enter(supermarket_queue)# Enter the supermarket queue for monitoring who is in the supermarket
        self.start_shopping()
        for next_product in self.route:
            if self.shopping_list[next_product]>0:
                self.move_to(next_product) #animation
                self.get_product(next_product)   
        self.go_to_checkout()
        self.leave(supermarket_queue)# Leave the supermarket queue
            
    def start_shopping(self):
        """ Get either a shopping cart or basket"""
        want_to_carry = cart_basket_distribution.sample()
        self.log_action(f"Entered cart/basket queue for {want_to_carry}")
        self.request(want_to_carry)
        self.move_to('got_cart')
        customer_basketcart_distribution_monitor.tally(want_to_carry)
        self.log_action(f"Got {want_to_carry}")
        self.carrying = want_to_carry
        
    def go_to_checkout(self):
        """Proceed to the emptiest queue in the checkout and wait while items are processed. Returns shopping cart/basket afterwards."""
        #enter emptiest queue
        emptiest_queue = min(checkouts, key=lambda checkout: checkout.requesters().length()) 
        self.move_to( [100+(15+checkouts.index(emptiest_queue)*2)*40, 100+1*40 + 10*emptiest_queue.requesters().length()]) #practically useless as it will snap to the shortest queue right after this.
        self.log_action(f"Entered checkout queue {emptiest_queue}")
        self.request(emptiest_queue) 
        self.log_action(f"Started checking out")
        item_scan_time = sum(time_per_item_distribution.sample() for _ in range(sum(self.shopping_list.values())))
        self.hold(item_scan_time+payment_time_distribution.sample()) #hold the customer for scanning all items and during payment
        self.log_action(f"Finished checking out")
        #return cart/basket (implicit since process finishes)    
        self.move_to([100+16*40,100 ])
        
        #print log if we want to debug
        if customer_to_log:
            if self.name() == f"customer.{customer_to_log}":
                print(self.carrying.claimers().print_info())
                print(f"Customer's Action Log for customer {self.name()}:")
                for time, action in self.actions_log:
                    print(f"At time {time}, customer: {action}")
        self.hold(till = self.traj.t1()) #wait till trajectory is finished before removing self from animation
        self.animation_representation.remove()
        
    def get_product(self, product):
        """
        Function to get the required number of {product}.
        Customer holds while the products are taken.
        Special cases for cheese_and_dairy and bread as they require clerks.
        """
        # For animation of department customers
        self.enter(departments[product])
        
        if product == "cheese_and_dairy":
            self.log_action(f"requesting cheese and dairy")
            self.request(cheese_and_dairy_clerks)
            self.log_action(f"Being helped for cheese and dairy")
            self.hold(cheese_and_dairy_time_distribution.sample())
            self.log_action(f"Got cheese and dairy")
            self.release(cheese_and_dairy_clerks)
        elif product == "bread":
            self.log_action(f"requesting bread")
            self.request(bread_clerks)
            self.log_action(f"Being helped for bread")
            self.hold(bread_time_distribution.sample())
            self.log_action(f"Got bread")
            self.release(bread_clerks)
        else:
            amount = self.shopping_list[product]
            self.log_action(f"Getting {product}")
            for _ in range(amount):
                self.hold(time_per_item_distribution.sample())
                self.log_action(f"Got {product}")
        
        # For animation of department customers
        self.leave(departments[product])

#customer generation
customer_distribution = [30, 80, 110, 90, 80, 70, 80, 90, 100, 120, 90, 40] #Expected total = 980
for index, customer_count  in enumerate(customer_distribution):
    env.ComponentGenerator(Customer, iat=env.Exponential(3600/customer_count), at=index*60*60, duration=60*60) #assumes time in seconds

#Checkout creation
checkouts = []
for i in range(number_of_checkouts):
    checkouts.append(env.Resource(f"checkout_clerk{i}", capacity = 1)) #3, 1.1s per item avg. payment 40-60s

#create store layout
sim.AnimateRectangle(spec=(100,100, 900, 700), fillcolor='', linecolor='black') #1 unit is 5 cm
sim.AnimateRectangle(spec=(100, 100+8*40, 100+2*40, 100+15*40), fillcolor='', linecolor='black') #Cheese
sim.AnimateRectangle(spec=(100,100, 100+2*40,100+8*40), fillcolor='', linecolor='black') #Bread
sim.AnimateRectangle(spec=(100+2*40, 100+2*40, 100+2*40, 100+12*40 ), fillcolor='', linecolor='black') #Meat and fish
for i in range(3):
    sim.AnimateRectangle(spec=(100+(3+3.5*i)*40, 100+2*40,100+(3+3+3.5*i)*40, 100+2.5*40), fillcolor='', linecolor='red', layer =1) #meat and fish aisles
sim.AnimateRectangle(spec=(100+2*40, 100+2*40, 100+14*40, 100+8*40), fillcolor='', linecolor='black') #Fruit and veggies
for i in range(3):
    sim.AnimateRectangle(spec=(100+(3+3.5*i)*40, 100+4*40,100+(3+3+3.5*i)*40, 100+6*40), fillcolor='', linecolor='green', layer =-1) #Fruit and veggies aisles
sim.AnimateRectangle(spec=(100+2*40, 100+2*40, 100+14*40, 100+15*40), fillcolor='', linecolor='black') #Canned food
for i in range(10):
    sim.AnimateRectangle(spec=(100+(3.25+i)*40, 100+8*40, 100+(3.25+0.5+i)*40, 100+14*40), fillcolor='', linecolor='gray', layer =-1) #Canned food aisles
sim.AnimateRectangle(spec=(100+14*40, 100+11*40, 100+20*40, 100+15*40), fillcolor='', linecolor='black') #Frozen food
for i in range(5):
    sim.AnimateRectangle(spec=(-(100-550+(14.5+i)*40), -(100-400+11*40), -(100-550+(15+i)*40), -(100-400+13.5*40)), angle=-10, fillcolor='', linecolor='blue', xy_anchor='ne', layer =-1) #Frozen food aisle. Messy because rotation is applied after translation.
sim.AnimateRectangle(spec=(100+14*40, 100+5*40, 100+20*40, 100+11*40), fillcolor='', linecolor='black') #Drinks
for i in range(6):
    sim.AnimateRectangle(spec=(100+(14+i)*40, 100+6*40, 100+(14.5+i)*40, 100+11*40), fillcolor='', linecolor='orange', layer =-1) #Drinks
sim.AnimateRectangle(spec=(100+14*40, 100+0*40, 100+20*40, 100+5*40), fillcolor='', linecolor='black') #Checkout
sim.AnimateRectangle(spec=(100+12*40,100+0,100+14*40, 100+2*40), fillcolor='', linecolor='black') #Carts and baskets
sim.AnimateRectangle(spec=(100+12*40,100+2*40,100+14*40, 100+2.5*40), fillcolor='white', linecolor='black') #Entry

#Routing information
routing_options ={ #Entering queues excluded as they are a special case in the move_to function
    "entry": [100+13*40, 100, 100+13*40, 100+1*40], #enter store at bottom, move up to carts
    "got_cart": [100+10*40, 100+2*40], #once you have the cart/basket move forward into store
    "fruit_and_vegetables": [100+6*40, 100+3*40], 
    "meat_and_fish": [100+6*40, 100+1*40, 100+3*40, 100+1*40],
    "canned_and_packed_food": [100+2.5*40, 100+14*40, 100+40*13, 100+40*14],
    "frozen_foods": [100+14*40, 100+12*40, 100+40*19, 100+40*11],
    "drinks": [100+19*40, 100+6.5*40],
}

#Queue animation

for index, q in enumerate(checkouts):
    sim.AnimateText(text=lambda q=q, index=index: f'C{index}:{q.requesters().length()+q.requesters().length()}', x=90+(15+index*2)*40, y=60+1*40)
    # checkout_trajectory = sim.TrajectoryPolygon(polygon = (90+(15+index*2)*40, 60+1*40, 100+(15+index*2)*40, 100+4*40, ), t0=0) 
                           # + sim.TrajectoryCircle(radius=10, x_center=100+(15+index*2)*40-10, y_center=100+4*40, angle0=90, angle1=0) + sim.TrajectoryPolygon(polygon = (100+(15+index*2)*40-10, 100+4*40+10, 100+(10)*40-10, 100+4*40+10, ))
    sim.AnimateQueue(q.requesters(), x=100+(15+index*2)*40, y=100+1*40, title='', direction='n', )#trajectory=checkout_trajectory
    
sim.AnimateQueue(bread_clerks.requesters(), x=100+2.5*40, y=100+8*40, direction = 's', title='')
sim.AnimateQueue(cheese_and_dairy_clerks.requesters(), x=100+2.5*40, y=100+15*40, direction = 's', title='')
 
#sim.AnimateMonitor(customer_basketcart_distribution_monitor)
sim.AnimateMonitor(supermarket_queue.length, x=100, y=0, horizontal_scale=1, vertical_scale=0.65, title = "Customers Population")
sim.AnimateText(text=lambda: f'Current: {supermarket_queue.length()}', x=305, y=60)
sim.AnimateText(text=lambda: f'Average: {round(supermarket_queue.length.mean(ex0=True)) if not math.isnan(supermarket_queue.length.mean(ex0=True)) else "N/A"}', x=305, y=30)
sim.AnimateText(text=lambda: f'Maximum: {supermarket_queue.length.maximum()}', x=305, y=0)


sim.AnimateMonitor(supermarket_queue.length_of_stay, x=500, y=0, horizontal_scale=1, vertical_scale=0.015, title = "Length of Stay")
# sim.AnimateText(text=lambda: f'Last LoS: {round(supermarket_queue.length_of_stay.get(), 2)} seconds', x=700, y=60)
sim.AnimateText(text=lambda: f'Average: {round(supermarket_queue.length_of_stay.mean(), 2)} seconds', x=705, y=60)
sim.AnimateText(text=lambda: f'Maximum: {round(supermarket_queue.length_of_stay.maximum(), 2)} seconds', x=705, y=30)
sim.AnimateText(text=lambda: f'Minimum: {round(supermarket_queue.length_of_stay.minimum(), 2)} seconds', x=705, y=0)

# Coordinates found through trial and error
department_monitor_positions = {
    "fruit_and_vegetables": (285, 355),
    "meat_and_fish": (275, 105),
    "bread": (102, 369),
    "cheese_and_dairy": (102, 530),
    "canned_and_packed_food": (190, 680),
    "frozen_foods": (660, 680),
    "drinks": (660, 305),
}

for department, queue in departments.items():
    x, y = department_monitor_positions[department]
    
    if queue.name() == "Bread":
        sim.AnimateText(text=lambda q=queue: f'{q.name()}\ncustomers:\n{q.length()}', x=x, y=y)
    elif queue.name() == "Cheese and Dairy":
        sim.AnimateText(text=lambda q=queue: f'{q.name().split(" ")[0]}\ncustomers:\n{q.length()}', x=x, y=y)
    else:
        sim.AnimateText(text=lambda q=queue: f'{q.name()} customers: {q.length()}', x=x, y=y)



In [2]:
# # Run for the full day (duration is in seconds) + 1 hour to make sure all customers can leave the store.
# run_count = 1
# for i in range(run_count):
#     env.run(sim.inf)
#     #get statistics
#     sim.reset() 

In [3]:
with env.video('fast_video.mp4'):

    env.run(50000)

Queue 0x19338ef8c10
  name=claimers of carts
  component(s):
    customer.499         enter_time 23228.366 priority=inf
    customer.500         enter_time 23338.386 priority=inf
    customer.501         enter_time 23382.578 priority=inf
    customer.503         enter_time 23434.922 priority=inf
    customer.504         enter_time 23512.295 priority=inf
    customer.505         enter_time 23516.160 priority=inf
    customer.506         enter_time 23517.753 priority=inf
    customer.508         enter_time 23687.150 priority=inf
    customer.510         enter_time 23725.447 priority=inf
    customer.512         enter_time 23786.561 priority=inf
    customer.513         enter_time 23815.984 priority=inf
    customer.514         enter_time 23984.613 priority=inf
    customer.516         enter_time 24061.646 priority=inf
    customer.517         enter_time 24096.565 priority=inf
None
Customer's Action Log for customer customer.500:
At time 23338.385539272233, customer: Entered cart/basket q

SimulationStopped: 